In [1]:
# wd = '/mnt/openfact/users/msawinski/factue-task2'
wd = '/Users/marcinsawinski/Documents/GitHub/factue-task2'
import sys, os
os.chdir(wd)

In [2]:
from factue.methods.llm_calls import load_template_list
for x in load_template_list(job='persuasion', step="detect", prompt_version='v001').keys():
    print(f'"{x}"')

"Name_Calling-Labeling"
"Doubt"
"Questioning_the_Reputation"
"Appeal_to_Fear-Prejudice"
"Appeal_to_Values"
"Exaggeration-Minimisation"
"Loaded_Language"
"Repetition"
"new"


In [3]:
import pandas as pd
from pathlib import Path
import os
root = Path("data/llm_output/persuasion")
# Example path template
files = root.rglob("*/*.parquet")

# Read all files and add file path as a column
df_list = []
for f in files:
    df_part = pd.read_parquet(f)
    df_part['source_file'] = f  # add the file path
    df_list.append(df_part)

# Combine into one DataFrame
df = pd.concat(df_list, ignore_index=True)
df['split'] = df.source_file.astype(str).str.split('/',expand=True)[7]

In [4]:
df.head(3)

,filename,start,end,text_lang,text,label_bin,label_multi,base_split,Questioning_the_Reputation,Name_Calling-Labeling,...,is_valid,extra_content,raw,think_content,extra_properties,pred,gold,source_file,error,split
0,pl_eu_12_06_2024_n01.txt,204,572,PL,Panie Marszałku! Wysoka Izbo! W dniu 22 maja\n...,False,[],train,[],[],...,[True],[None],"[{""description"": ""No, the Exaggeration – Minim...",[None],[None],0,0,data/llm_output/persuasion/detect/GPT_4O_MINI/...,NaN,dev
1,pl_abortion_11_04_2024_n01.txt,1032,1828,PL,Panie Marszałku! Wysoka Izbo! Dlaczego kobiety...,True,"[Conversation_Killer, Appeal_to_Pity, Appeal_t...",trial,[],[],...,[True],[None],"[{\n ""description"": ""The Exaggeration - Minim...",[None],[None],0,0,data/llm_output/persuasion/detect/GPT_4O_MINI/...,NaN,dev
2,pl_eu_12_06_2024_n02.txt,3287,3570,PL,Wicemarszałek Włodzimierz Czarzasty:\nDziękuję...,False,[],train,[],[],...,[True],[None],"[{""description"": ""No, the Exaggeration – Minim...",[None],[None],0,0,data/llm_output/persuasion/detect/GPT_4O_MINI/...,NaN,dev


In [5]:
df.pred.value_counts()

pred
 0    172
 1    128
-1     10
Name: count, dtype: int64

In [13]:
df[df.split=='dev'].groupby(["prompt_name","text_lang","model_name","split"])['filename'].agg("count").reset_index()

,prompt_name,text_lang,model_name,split,filename
0,Appeal_to_Fear-Prejudice,BG,gpt-4o-mini,dev,10
1,Appeal_to_Fear-Prejudice,PL,gpt-4o-mini,dev,10
2,Appeal_to_Fear-Prejudice,RU,gpt-4o-mini,dev,10
3,Appeal_to_Fear-Prejudice,SI,gpt-4o-mini,dev,10
4,Appeal_to_Values,BG,gpt-4o-mini,dev,10
5,Appeal_to_Values,PL,gpt-4o-mini,dev,10
6,Appeal_to_Values,RU,gpt-4o-mini,dev,10
7,Appeal_to_Values,SI,gpt-4o-mini,dev,10
8,Doubt,BG,gpt-4o-mini,dev,10
9,Doubt,PL,gpt-4o-mini,dev,10


In [7]:
def normalize_binary(x):
    return 1 if str(x).strip().lower() in {'1', 'true'} else 0

In [8]:
df['source_file'].value_counts().sort_index()
df['gold'] = df['gold'].apply(normalize_binary)
df['pred'] = df['pred'].apply(normalize_binary)
df['split'] = df.source_file.astype(str).str.split('/',expand=True)[7]

In [9]:
df.prompt_name.value_counts()

prompt_name
Exaggeration-Minimisation     40
Repetition                    40
Appeal_to_Values              40
Appeal_to_Fear-Prejudice      40
Questioning_the_Reputation    40
Loaded_Language               40
Doubt                         40
Name_Calling-Labeling         30
Name: count, dtype: int64

In [10]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
# Define a function to compute metrics for a group
# Assume df has columns: prompt_name, text_lang, gold, pred
rows = []

def is_valid_label(x):
    return x in (0, 1)

rows = []

for prompt_name in df['prompt_name'].unique():
    for text_lang in df.loc[df['prompt_name'] == prompt_name, 'text_lang'].unique():
        for model_name in df.loc[(df['prompt_name'] == prompt_name) & (df['text_lang'] == text_lang), 'model_name'].unique():
            group = df[
                (df['prompt_name'] == prompt_name) &
                (df['text_lang'] == text_lang) &
                (df['model_name'] == model_name)
            ]
            if len(group) > 0:
                row = {
                    'prompt_name': prompt_name,
                    'text_lang': text_lang,
                    'model_name': model_name,
                    'accuracy': accuracy_score(group['gold'], group['pred']),
                    'precision': precision_score(group['gold'], group['pred'], zero_division=0),
                    'recall': recall_score(group['gold'], group['pred'], zero_division=0),
                    'f1': f1_score(group['gold'], group['pred'], zero_division=0),
                    'support': len(group)
                }
                rows.append(row)

results = pd.DataFrame(rows)
results[['accuracy', 'precision','recall', 'f1']] = results[['accuracy', 'precision','recall', 'f1']].round(2)
results

,prompt_name,text_lang,model_name,accuracy,precision,recall,f1,support
0,Exaggeration-Minimisation,PL,gpt-4o-mini,0.4,0.00,0.0,0.00,10
1,Exaggeration-Minimisation,RU,gpt-4o-mini,0.5,0.00,0.0,0.00,10
2,Exaggeration-Minimisation,SI,gpt-4o-mini,0.9,0.00,0.0,0.00,10
3,Exaggeration-Minimisation,BG,gpt-4o-mini,0.6,0.25,0.5,0.33,10
4,Repetition,PL,gpt-4o-mini,0.6,0.43,1.0,0.60,10
5,Repetition,RU,gpt-4o-mini,0.8,0.00,0.0,0.00,10
6,Repetition,SI,gpt-4o-mini,0.5,0.00,0.0,0.00,10
7,Repetition,BG,gpt-4o-mini,0.6,0.20,1.0,0.33,10
8,Appeal_to_Values,PL,gpt-4o-mini,0.9,0.83,1.0,0.91,10
9,Appeal_to_Values,RU,gpt-4o-mini,0.7,0.25,1.0,0.40,10


In [11]:
import seaborn as sns
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
fig, axes = plt.subplots(len(metrics), 1, figsize=(10, 18), sharex=True)

for i, metric in enumerate(metrics):
    ax = axes[i]
    sns.barplot(
        data=results,
        y='technique_id',
        x=metric,
        hue='text_lang',
        ax=ax,
        palette='muted',
        errorbar=None  # fallback for compatibility
    )
    ax.set_title(metric.capitalize())
    ax.set_xlim(0, 1.05)
    ax.set_ylabel('Technique ID')
    ax.set_xlabel('Score')
    ax.legend(title='Text Language', loc='lower right')

plt.tight_layout()
plt.show()

In [ ]:
# # Create a melt of metrics for faceted visualization
# metrics_df = results.melt(
#     id_vars=['technique_id', 'text_lang'],
#     value_vars=['accuracy', 'precision', 'recall', 'f1'],
#     var_name='metric',
#     value_name='score'
# )

# # Plot
# plt.figure(figsize=(12, 6))
# sns.barplot(data=metrics_df, x='technique_id', y='score', hue='text_lang', palette='muted', errorbar=None)
# plt.title('Metrics by Technique and Language')
# plt.ylim(0, 1.05)
# plt.ylabel('Score')
# plt.xlabel('Technique ID')
# plt.legend(title='Text Language')
# plt.grid(axis='y', linestyle='--', alpha=0.7)
# plt.tight_layout()
# plt.show()

# total

In [ ]:
# Metrics
acc = accuracy_score(df['gold'], df['pred'])
precision = precision_score(df['gold'], df['pred'], zero_division=0)
recall = recall_score(df['gold'], df['pred'], average='binary')
f1 = f1_score(df['gold'], df['pred'], average='binary')

print(f"Accuracy: {acc:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1-score: {f1:.3f}")